# **Importing and setting up the enviroments**

In [ ]:
import pickle
import h5py
import seaborn as sns
import matplotlib
from sklearn.dummy import DummyClassifier
import Bio
import widis_lstm_tools
import torch
import ipywidgets

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 2.3MB 5.6MB/s 
  Cloning https://github.com/widmi/widis-lstm-tools to /tmp/pip-req-build-mrmj9oea
  Running command git clone -q https://github.com/widmi/widis-lstm-tools /tmp/pip-req-build-mrmj9oea
  Created wheel for widis-lstm-tools: filename=widis_lstm_tools-0.4-cp36-none-any.whl size=43228 sha256=db59e85d2bcc5574936d1aac7158b9c089466c974c243c34e2eb2b4f0f391258
  Stored in directory: /tmp/pip-ephem-wheel-cache-qr7s2zzq/wheels/a4/f3/2b/9a49ec2d9387e8c376e6ba7e02323b2657ba1980a483535a72
Successfully built widis-lstm-tools


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
with open('/content/drive/My Drive/Colab Notebooks/fullsequence.pkl', 'rb') as f:
    fulldata = pickle.load(f)
full_sequences = fulldata
with open('/content/drive/My Drive/Colab Notebooks/uni2go.pkl', 'rb') as f:
    # Pickle will store our object into the specified file
    uni_data = pickle.load(f)
with open('/content/drive/My Drive/Colab Notebooks/goid.pkl', 'rb') as f:
    # Pickle will store our object into the specified file
    ggid = pickle.load(f)
gid = h5py.File('/content/drive/My Drive/Colab Notebooks/one_hot_go.h5py','r')

ImportError: ignored

# **Preprocessing**

In [ ]:
import pickle
print(len(full_sequences))
b1 = len(full_sequences)*.01
b2 = len(full_sequences)*.99
print(b1,b2)
counts = []
e = []
classes =[]
import numpy as np


for value in ggid.values():
  if value>250:
    e.append(value)
print('mean:'+str(sum(e)/len(e)))
print('classes left from 30821 which have sequences connected to them :'+str(len(e)))
i = 0
for v in e:
  if v<=(sum(e)/len(e)):
    i+=1
print('classes under the mean or equal:'+str(i))
import matplotlib.pyplot as plt
for key in ggid.keys():
  if 600>ggid[key]>200:
    classes.append(key)
    counts.append(ggid[key])
print(len(classes))
print(len(counts))
from matplotlib import pyplot as plt
k = zip(classes,counts)
newggid = dict(k)
with open('newgoid.pkl', 'wb') as f:
    pickle.dump(newggid, f)
    print('done')
print(len(newggid.keys()))
print(set(classes))

In [ ]:
%matplotlib inline
print(counts)
plt.figure(figsize=(20,10))
plt.hist(counts)
plt.title("Annotations in each goid",fontsize=20)
plt.xticks(fontsize= 10)
plt.yticks(fontsize= 10)
plt.xlabel('GO-ID',fontsize=20)
plt.ylabel('Occurences',fontsize=20)
plt.show()
plt.savefig('data')

In [ ]:

top_10_idx = np.argsort(list(ggid.values()))[-10:]
top_10_values = [list(ggid.values())[i] for i in top_10_idx]
top_10_keys = [list(ggid.keys())[i] for i in top_10_idx]

print('top ten classes which have the most sequences=',top_10_keys)
top_10_values


In [ ]:
from sklearn.dummy import DummyClassifier
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, recall_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

import pandas as pd

In [ ]:
datadict={}
for key in fulldata.keys():
  datadict[fulldata[key]] = list(uni_data[key])



In [ ]:
binarizerlist = []
for key in newggid.keys():
  g = []
  for values in datadict.values():
    if key in values:
      g.append(1)
    else:
      g.append(0)
  binarizerlist.append(g)
len(binarizerlist)


In [ ]:
import numpy as np 
vocab = 'MAFSEDVLKYRPNWQCGIHTXZBUO'
aa_lookup = dict([(k, v) for v, k in enumerate(vocab)])
bin_seq = []
for sequence in datadict.keys():  
  x = np.zeros(shape=(len(sequence), len(aa_lookup)), dtype=np.float32)
  x[np.arange(len(sequence)), [aa_lookup[aa] for aa in sequence]] = 1
  print(len(x))
  bin_seq.append(x)


In [ ]:
columnst = {}
columnst['sequences'] = bin_seq
for key in newggid.keys():
  columnst[str('GO:'+str(key))] = binarizerlist[list(newggid.keys()).index(key)]
columnst.keys()

In [ ]:
from tqdm import tqdm_notebook as tqdms
# Assign data to tuples.  
  
  
# Converting lists of tuples into  
# pandas Dataframe.  
df = pd.DataFrame(columnst)  
     
# Print data.
df




In [ ]:
categories = list(df.columns.values)[1:]
sns.set(font_scale = .05)
plt.figure()

print('1')
ax= sns.barplot(categories, df.iloc[:].sum().values)
plt.title("Annotations in each goid", fontsize=24)
plt.ylabel('Number of Annotations', fontsize=18)
plt.xlabel('GoID', fontsize=18)
#adding the text labels
print('2')
rects = ax.patches
labels = df.iloc[:].sum().values
print('3')
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom', fontsize=8)
plt.show()


In [ ]:
newdf = df[df.iloc[:,1:].sum(axis=1)>0]
newdf
rowSums = newdf.iloc[1:,:].sum(axis=1)
multiLabel_counts = rowSums.value_counts()
multiLabel_counts = multiLabel_counts.iloc[:]
print(multiLabel_counts.values)
sns.set(font_scale = 2)
plt.figure(figsize=(15,8))
ax = sns.barplot(multiLabel_counts.index, multiLabel_counts.values)
plt.title("sequences having multiple labels ")
plt.ylabel('Number of sequences', fontsize=18)
plt.xlabel('Number of labels', fontsize=18)
#adding the text labels
rects = ax.patches
labels = multiLabel_counts.values
for rect, label in zip(rects, labels):
    height = rect.get_height()
    ax.text(rect.get_x() + rect.get_width()/2, height + 5, label, ha='center', va='bottom')
plt.show()

In [ ]:
multiLabel_counts.values
rowSums.value_counts()

In [ ]:
sns.distplot(classes,)

#Classifier Chain

In [ ]:
import sklearn
from  sklearn.metrics import balanced_accuracy_score,auc,f1_score,roc_curve
from  sklearn.ensemble import RandomForestClassifier 

X = df['sequences']
Y = df.drop('sequences',axis=1)
x_train, x_test,y_train,y_test =train_test_split(X, Y, test_size=0.2, random_state=42) 
newlist = []
auclist = []
bacclist = []
f1_list = []
for i in Y.keys():
    task = str(i)
    y_train = y_train[task]
    x_trains = x_train
    print(y_train,x_trains)
    RN = RandomForestClassifier(random_state = 3,n_jobs=60, n_estimators=200)
    RN.fit(x_trains,y_train)
    v = RN.predict(y_test)
    fpr, tpr, thresholds = metrics.roc_curve(y_test[i], v)
    aucs = auc(fpr, tpr)
    baccur = balanced_accuracy_score(y_test,v)
    f1 = f1_score(y_test,v)
    auclist.append(aucs)
    newlist.append(RN)
    bacclist.append(baccur)
    f1_list.append(f1)
print(len(auclist),len(newlist))#check if the list are the same size, for the next step 
print(auclist)

    

In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(classes)
print(y)

In [ ]:
classes = []
labels = []
go_matrix = gid['one_hot_dag'][:]
goid_to_index = gid['go_id_to_index'][:]
allowed_goids = np.array(list(ggid.values()))>-1
label = np.zeros_like(go_matrix[0], dtype=np.float32).flatten()
for key in datadict.keys():
  for go_id in datadict[key]:
    goid_to_idx = goid_to_index[go_id]
    if goid_to_idx >= 0:
      label += go_matrix[goid_to_idx]
    label[:] = label > 0
    label = label[allowed_goids]
    classes.append(label)
  
print(len(classes))
print(len(sequences))


In [ ]:
classes = []
sequences = []
for key in datadict.keys():
  classes= classes+datadict[key]
  sequences.extend([key]*len(datadict[key]))
  
print(len(classes))
print(len(sequences))



In [ ]:
len(classes)

chose the samples which only have one label to make it easier then choose the multilabel ones later


In [ ]:
grouped_tags = df.groupby("GOID", sort='count').size().reset_index(name='count')
fig = plt.figure()
grouped_tags.plot(title="Tag frequency")


In [ ]:
fulldataset = pd.DataFrame(bin_seq,classes,columns = ['Sequence','Connected classes'])

In [ ]:
y = fulldataset['Connected classes']
X = fulldataset['Sequence'] 


In [ ]:
# DummyClassifier to predict only target 0
dummy = DummyClassifier(strategy='most_frequent').fit(X_train, y_train)
dummy_pred = dummy.predict(X_test)
print(dummy_pred)
# checking unique labels
print('Unique predicted labels: ', (np.unique(dummy_pred)))

# checking accuracy
print('Test score: ', accuracy_score(y_test, dummy_pred))

In [ ]:

labels = []
go_matrix = gid['one_hot_dag'][:]
goid_to_index = gid['go_id_to_index'][:]
allowed_goids = np.array(list(ggid.values()))>-1
label = np.zeros_like(go_matrix[0], dtype=np.float32).flatten()
for go_id in classes:
  goid_to_idx = goid_to_index[go_id]
  if goid_to_idx >= 0:
    label += go_matrix[goid_to_idx]
  label[:] = label > 0
  label = label[allowed_goids]
  labels.append(label)
  import numpy as np

classes = []
labels = []
go_matrix = gid['one_hot_dag'][:]
goid_to_index = gid['go_id_to_index'][:]
allowed_goids = np.array(list(ggid.values()))>-1
label = np.zeros_like(go_matrix[0], dtype=np.float32).flatten()
for key in datadict.keys():
  for go_id in datadict[key]:
    goid_to_idx = goid_to_index[go_id]
    if goid_to_idx >= 0:
      label += go_matrix[goid_to_idx]
    label[:] = label > 0
    label = label[allowed_goids]
    classes.append(label)
  
print(len(classes))
print(len(sequences))